In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import warnings

In [2]:
df = pd.read_csv("train.csv", sep = ",")
print(df.shape)
df.head()

(35344, 11)


,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
sample = df.copy()

In [39]:
sample.loc[:, sample.columns != 'demand'] = sample.loc[:, sample.columns != 'demand'].fillna(0)

In [40]:
sample.isnull().any()

Unnamed: 0           False
product_rk           False
store_location_rk    False
period_start_dt      False
demand                True
PROMO1_FLAG          False
PROMO2_FLAG          False
PRICE_REGULAR        False
PRICE_AFTER_DISC     False
NUM_CONSULTANT       False
AUTORIZATION_FLAG    False
dtype: bool

In [41]:
import datetime
sample['period_start_dt']= pd.to_datetime(sample['period_start_dt']) 
sample.dtypes

Unnamed: 0                    int64
product_rk                    int64
store_location_rk             int64
period_start_dt      datetime64[ns]
demand                      float64
PROMO1_FLAG                 float64
PROMO2_FLAG                 float64
PRICE_REGULAR               float64
PRICE_AFTER_DISC            float64
NUM_CONSULTANT              float64
AUTORIZATION_FLAG           float64
dtype: object

In [6]:
sample

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,1,40370,309,2016-12-19,64.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,2,40372,309,2016-12-19,32.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,3,40373,309,2016-12-19,10.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,4,46272,309,2016-12-19,15.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
35309,35507,40370,1380,2019-11-25,24.0,0.0,0.0,1000.000000,1000.000000,0.0,1.0
35310,35508,40372,1380,2019-11-25,11.0,0.0,0.0,2000.000000,2000.000000,0.0,1.0
35311,35509,40373,1380,2019-11-25,3.0,0.0,0.0,3000.000000,3000.000000,0.0,1.0
35312,35510,46272,1380,2019-11-25,0.0,1.0,0.0,284.290000,199.000000,0.0,1.0


In [42]:
sample['month'] = sample['period_start_dt'].dt.month
sample.head()

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,month
0,0,40369,309,2016-12-19,29.0,0.0,0.0,0.0,0.0,0.0,0.0,12
1,1,40370,309,2016-12-19,64.0,0.0,0.0,0.0,0.0,0.0,0.0,12
2,2,40372,309,2016-12-19,32.0,0.0,0.0,0.0,0.0,0.0,0.0,12
3,3,40373,309,2016-12-19,10.0,0.0,0.0,0.0,0.0,0.0,0.0,12
4,4,46272,309,2016-12-19,15.0,0.0,0.0,0.0,0.0,0.0,0.0,12


In [43]:
seasons = ['winter', 'winter', 'spring', 'spring', 'spring', 'summer', 'summer', 'summer', 
           'autumn', 'autumn', 'autumn', 'winter']
month_to_season = dict(zip(range(1,13), seasons))
month_to_season 

{1: 'winter',
 2: 'winter',
 3: 'spring',
 4: 'spring',
 5: 'spring',
 6: 'summer',
 7: 'summer',
 8: 'summer',
 9: 'autumn',
 10: 'autumn',
 11: 'autumn',
 12: 'winter'}

In [44]:
sample['season'] = sample['month'].map(month_to_season) 

In [45]:
one_hot = pd.get_dummies(sample['season'])
sample = sample.drop('season', axis = 1)
sample = sample.join(one_hot)
sample

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,month,autumn,spring,summer,winter
0,0,40369,309,2016-12-19,29.0,0.0,0.0,0.00,0.0,0.0,0.0,12,0,0,0,1
1,1,40370,309,2016-12-19,64.0,0.0,0.0,0.00,0.0,0.0,0.0,12,0,0,0,1
2,2,40372,309,2016-12-19,32.0,0.0,0.0,0.00,0.0,0.0,0.0,12,0,0,0,1
3,3,40373,309,2016-12-19,10.0,0.0,0.0,0.00,0.0,0.0,0.0,12,0,0,0,1
4,4,46272,309,2016-12-19,15.0,0.0,0.0,0.00,0.0,0.0,0.0,12,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0,12,0,0,0,1
35340,35538,40372,1380,2019-12-30,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0,12,0,0,0,1
35341,35539,40373,1380,2019-12-30,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0,12,0,0,0,1
35342,35540,46272,1380,2019-12-30,NaN,1.0,0.0,284.29,199.0,0.0,1.0,12,0,0,0,1


In [46]:
sample = sample.sort_values(by=['period_start_dt', 'product_rk', 'store_location_rk'])

In [47]:
for i in [7, 14, 28]:
    sample["price_reg_lag_{}".format(i)] = sample.PRICE_REGULAR.shift(i, fill_value=0)
    sample["price_after_lag_{}".format(i)] = sample.PRICE_AFTER_DISC.shift(i, fill_value=0)

In [48]:
sample

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,...,autumn,spring,summer,winter,price_reg_lag_7,price_after_lag_7,price_reg_lag_14,price_after_lag_14,price_reg_lag_28,price_after_lag_28
0,0,40369,309,2016-12-19,29.0,0.0,0.0,0.000000,0.000000,0.0,...,0,0,0,1,0.00,0.0,0.000000,0.000000,0.000000,0.000000
15,15,40369,317,2016-12-19,50.0,0.0,0.0,0.000000,0.000000,0.0,...,0,0,0,1,0.00,0.0,0.000000,0.000000,0.000000,0.000000
938,944,40369,355,2016-12-19,38.0,0.0,0.0,0.000000,0.000000,0.0,...,0,0,0,1,0.00,0.0,0.000000,0.000000,0.000000,0.000000
1861,1873,40369,380,2016-12-19,39.0,0.0,0.0,0.000000,0.000000,0.0,...,0,0,0,1,0.00,0.0,0.000000,0.000000,0.000000,0.000000
2784,2802,40369,425,2016-12-19,105.0,0.0,0.0,0.000000,0.000000,0.0,...,0,0,0,1,0.00,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32975,33150,96212,1326,2019-12-30,NaN,1.0,0.0,99.000000,69.300000,0.0,...,0,0,0,1,141.43,99.0,141.430000,99.000000,141.430000,99.000000
33701,33881,96212,1328,2019-12-30,NaN,1.0,0.0,121.021429,84.714286,0.0,...,0,0,0,1,70.00,49.0,99.000000,69.300000,0.000000,0.000000
34349,34535,96212,1347,2019-12-30,NaN,1.0,0.0,141.430000,99.000000,0.0,...,0,0,0,1,141.43,99.0,141.430000,99.000000,100.612857,70.428571
35018,35210,96212,1363,2019-12-30,NaN,1.0,0.0,141.430000,99.000000,0.0,...,0,0,0,1,0.00,0.0,121.021429,84.714286,0.000000,0.000000


In [49]:
for i in [7, 14, 28]:
    sample[f'SMA_price_reg_{i}'] = sample.PRICE_REGULAR.rolling(i, min_periods = 1).mean()
    sample[f'SMA_price_after_{i}'] = sample.PRICE_AFTER_DISC.rolling(i, min_periods = 1).mean()

In [50]:
for i in [7, 14, 28]:
    sample[f'STD_price_reg_{i}'] = sample.PRICE_REGULAR.rolling(i, min_periods = 1).std()
    sample[f'STD_price_after_{i}'] = sample.PRICE_AFTER_DISC.rolling(i, min_periods = 1).std()

In [51]:
sample['PRICE_DIF'] = sample.PRICE_REGULAR - sample.PRICE_AFTER_DISC

In [52]:
sample = sample.drop('month', axis=1)

In [53]:
sample.loc[:, sample.columns != 'demand'] = sample.loc[:, sample.columns != 'demand'].fillna(0)

### cо спросом

In [107]:
samp = df[df['demand'].isnull() == False]

In [108]:
target = samp[round(samp.shape[0] * 0.8):].demand

In [110]:
samp = samp.fillna(0)

In [111]:
samp[round(samp.shape[0] * 0.8):].demand = np.zeros(6829)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [112]:
samp[round(samp.shape[0] * 0.8):].demand = samp[round(samp.shape[0] * 0.8):].demand.replace(0, np.nan)

In [116]:
samp['period_start_dt']= pd.to_datetime(samp['period_start_dt']) 

In [117]:
samp['month'] = samp['period_start_dt'].dt.month
samp['season'] = samp['month'].map(month_to_season) 
one_hot = pd.get_dummies(samp['season'])
samp = samp.drop('season', axis = 1)
samp = samp.join(one_hot)


In [119]:
samp = samp.sort_values(by=['period_start_dt', 'product_rk', 'store_location_rk'])

In [121]:
for i in [7, 14, 28]:
    samp["price_reg_lag_{}".format(i)] = samp.PRICE_REGULAR.shift(i, fill_value=0)
    samp["price_after_lag_{}".format(i)] = samp.PRICE_AFTER_DISC.shift(i, fill_value=0)
for i in [7, 14, 28]:
    samp[f'STD_price_reg_{i}'] = samp.PRICE_REGULAR.rolling(i, min_periods = 1).std()
    samp[f'STD_price_after_{i}'] = samp.PRICE_AFTER_DISC.rolling(i, min_periods = 1).std()

In [123]:
samp = samp.drop('month', axis=1)

In [124]:
samp['PRICE_DIF'] = samp.PRICE_REGULAR - sample.PRICE_AFTER_DISC

In [129]:
for i in [7, 14, 28, 51]:
    samp["demand_lag_{}".format(i)] = samp.demand.shift(i, fill_value=0)
for i in [7, 14, 28, 51]:
    samp[f'STD_demand_{i}'] = samp.demand.rolling(i, min_periods = 1).std()
for i in [7, 14, 51]:
    samp[f'SMA_demand_{i}'] = samp.demand.rolling(i, min_periods = 1).mean()

In [145]:
samp.groupby('product_rk').mean()['demand']

product_rk
40369    16.763146
40370    21.778185
40372     9.010167
40373     5.865710
46272     6.514788
96212     7.617146
Name: demand, dtype: float64

In [165]:
samp = samp.drop('mean_demand', axis=1)

In [171]:
mean_demand = samp.groupby(by='product_rk').mean()['demand'].tolist()
mean_price_reg = samp.groupby(by='product_rk').mean()['PRICE_REGULAR'].tolist()
mean_price_after = samp.groupby(by='product_rk').mean()['PRICE_AFTER_DISC'].tolist()
for i in enumerate([40369, 40370, 40372, 40373, 46272, 96212]):
    samp.loc[samp['product_rk'] == i[1], 'mean_demand'] = mean_demand[i[0]]
    samp.loc[samp['product_rk'] == i[1], 'mean_price_reg'] = mean_price_reg[i[0]]
    samp.loc[samp['product_rk'] == i[1], 'mean_price_after'] = mean_price_after[i[0]]

In [180]:
stores = samp.groupby(by='store_location_rk').mean()['demand'].index.tolist()
store_mean_demand = samp.groupby(by='store_location_rk').mean()['demand'].tolist()
store_mean_price_reg = samp.groupby(by='store_location_rk').mean()['PRICE_REGULAR'].tolist()
store_mean_price_after = samp.groupby(by='store_location_rk').mean()['PRICE_AFTER_DISC'].tolist()
for i in enumerate(stores):
    samp.loc[samp['store_location_rk'] == i[1], 'store_mean_demand'] = store_mean_demand[i[0]]
    samp.loc[samp['store_location_rk'] == i[1], 'store_mean_price_reg'] = store_mean_price_reg[i[0]]
    samp.loc[samp['store_location_rk'] == i[1], 'store_mean_price_after'] = store_mean_price_after[i[0]]

In [184]:
samp.loc[:, samp.columns != 'demand'] = samp.loc[:, samp.columns != 'demand'].fillna(0)

In [194]:
X_train = samp[samp['demand'].isnull() == False].drop('demand', axis=1).set_index('period_start_dt')
y_train = samp[samp['demand'].isnull() == False].demand
X_test = samp[samp['demand'].isnull() == True].drop('demand', axis=1).set_index('period_start_dt')
y_test = target

In [197]:
def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [199]:
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

In [202]:
rf = RandomForestRegressor(n_estimators = 50)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
smape = smape(y_test, pred)
print(f'N_est={i}, mae={mae}, mse={mse}, smape={smape}')

N_est=100, mae=16.059798073029725, mse=1848.893492215646, smape=141.7356890373391


In [ ]:
for i in tqdm(range(100, 1000, 100)):
    rf = RandomForestRegressor(n_estimators = i, random_state = 42)
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    print(f'N_est={i}, mae={mae}, mse={mse}')

 11%|█         | 1/9 [00:48<06:24, 48.08s/it]

N_est=100, mae=15.909268551780643, mse=1837.7632124252634


 22%|██▏       | 2/9 [02:22<07:13, 61.98s/it]

N_est=200, mae=15.896391273826328, mse=1834.0962831764282


 33%|███▎      | 3/9 [04:51<08:48, 88.02s/it]

N_est=300, mae=15.903460531263239, mse=1835.7181050331399


## проьа

In [63]:
X_train = sample[sample['demand'].isnull() == False].drop('demand', axis=1).set_index('period_start_dt')
y_train = sample[sample['demand'].isnull() == False].demand

In [64]:
X_test = sample[sample['demand'].isnull() == True].drop('demand', axis=1).set_index('period_start_dt')

In [72]:
X_test['Unnamed: 0']

period_start_dt
2019-12-02      908
2019-12-02     1837
2019-12-02     2766
2019-12-02     3696
2019-12-02     4625
              ...  
2019-12-30    33150
2019-12-30    33881
2019-12-30    34535
2019-12-30    35210
2019-12-30    35541
Name: Unnamed: 0, Length: 1200, dtype: int64

In [65]:
tree = DecisionTreeRegressor(max_depth=20, min_samples_leaf=30) 
tree.fit(X_train, y_train)
y_test_pred = tree.predict(X_test) 

In [74]:
res_1 = pd.DataFrame(X_test['Unnamed: 0'])

In [76]:
res_1['pred'] = y_test_pred

In [80]:
res_1.rename(columns={"Unnamed: 0": "id", "pred": "predicted"}).set_index('id').to_csv('Aleksandra_Romanenko_181_hw2_2.csv')

## Обучение

In [22]:
X_tr = sample[:round(sample.shape[0] * 0.8)].drop('demand', axis=1).set_index('period_start_dt')
y_tr = sample[:round(sample.shape[0] * 0.8)].demand

In [23]:
X_ts = sample[round(sample.shape[0] * 0.8):].drop(['demand'], axis=1).set_index('period_start_dt')
y_ts = sample[round(sample.shape[0] * 0.8):].demand

In [24]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [25]:
from sklearn.tree import DecisionTreeRegressor 
MAE = []
for i in [10, 15, 20, 25, 30]:
    for j in [5, 10, 15, 20, 25, 30]:
        
        tree = DecisionTreeRegressor(max_depth=i, min_samples_leaf=j) 
 
        tree.fit(X_tr, y_tr)
        y_ts_pred = tree.predict(X_ts) 

        mae = mean_absolute_error(y_ts, y_ts_pred)
        mse = mean_squared_error(y_ts, y_ts_pred)
        MAE.append(mae)
        print(f'Depth={i}, min_leaph={j}, mae={mae}, mse={mse}')

Depth=10, min_leaph=5, mae=5.670813307455569, mse=88.36360295075528
Depth=10, min_leaph=10, mae=5.909194778156548, mse=89.93987660713394
Depth=10, min_leaph=15, mae=5.772179572582704, mse=80.79861750324496
Depth=10, min_leaph=20, mae=5.664284972547212, mse=82.07548242838979
Depth=10, min_leaph=25, mae=5.645991183432576, mse=79.03886124065701
Depth=10, min_leaph=30, mae=5.726024803470648, mse=82.84570079719812
Depth=15, min_leaph=5, mae=5.796359607635907, mse=102.63882698867201
Depth=15, min_leaph=10, mae=6.140546201245233, mse=102.40021817223405
Depth=15, min_leaph=15, mae=5.708956832180092, mse=85.41144016767771
Depth=15, min_leaph=20, mae=5.662117786285795, mse=88.74459311856273
Depth=15, min_leaph=25, mae=5.433126163129811, mse=76.45931696861717
Depth=15, min_leaph=30, mae=5.72567287747039, mse=84.53380477861546
Depth=20, min_leaph=5, mae=6.087613840021297, mse=115.07444763109739
Depth=20, min_leaph=10, mae=6.234119069388347, mse=105.6880816649255
Depth=20, min_leaph=15, mae=5.81318

In [62]:
min(MAE)

5.433126163129811

Depth=25, min_leaph=25, mae=5.089076808244731

In [26]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_tr, y_tr)
y_lr_pred = lr.predict(X_ts)
mean_squared_error(y_ts, y_lr_pred)

128.2382792678516